In [2]:
import glob
import os
import json
import re

In [3]:
folder_name = "../bora-univ/"

#context list
context_name_list = os.listdir(folder_name + "ann.json/master/pool")

#relation 폴더 경로
relation_folder_paths = glob.glob(folder_name + "ann.json/master/pool/*")
relation_folder_paths = relation_folder_paths + glob.glob(folder_name + "ann.json/members/*")

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]


#anntation_lenged 정보
annotation_legend = folder_name + "annotations-legend.json"
with open(annotation_legend,"r",encoding='utf-8') as f:
    annotation_legend = json.load(f)

In [4]:
#relation dictionary 파일로부터 subject와 object의 entity 정보를 추출해주는 함수
def get_needed_relation_data(tmp_relation):
    subject_token = re.findall("\(+(.+)+\)",annotation_legend[tmp_relation["relations"][0]['classId']])[0].split("|")[0]
    if subject_token == tmp_relation['entities'][0]['classId']:
        sub_entity, obj_entity = tmp_relation['entities']
    else:
        obj_entity, sub_entity  = tmp_relation['entities']
    
    # get preprocessed entities
    def _get_entity(entity):
        outputs = entity['offsets'][0]
        outputs['type'] = annotation_legend[entity['classId']].split("-")[1].lower()
        return outputs
    
    output_subject = _get_entity(sub_entity)
    output_object = _get_entity(obj_entity)
    return output_subject, output_object

In [5]:
#relation dictionary 파일로부터 title(relation)을 추출해주는 함수
def get_label(relation_json):
    label_tag = relation_json['relations'][0]['classId'] #r_6
    try:
        _,sub_type, label = annotation_legend[re.findall("\(+(.+)+\|",annotation_legend[label_tag])[0]].split("-")
        return f"{sub_type}:{label}"
    except:
        _,sub_type, = annotation_legend[re.findall("\(+(.+)+\|",annotation_legend[label_tag])[0]].split("-")
        return f"{sub_type}:no_relation"

In [6]:
annotation_legend

{'r_99': 'f9kp5pvi7zl(e_76|e_97)',
 'r_114': 'uskwksnq5p0(e_40|e_106)',
 'r_67': '3ucbjvs4hq8(e_54|e_55)',
 'r_98': 'bvbuv4jete0(e_76|e_96)',
 'e_49': 'OBJ-LAN-상위_언어',
 'e_109': 'OBJ-STU-영향',
 'r_89': 'xqpqpf4cmj0(e_76|e_83)',
 'e_97': 'OBJ-NOH-관계_없음',
 'e_121': 'SUBJ-STU-요소',
 'e_82': 'OBJ-ORG-관계_없음',
 'r_60': 'm4idonqz378(e_40|e_41)',
 'e_105': 'OBJ-POH-별칭',
 'e_38': 'SUBJ-STU-상위_학문',
 'e_46': 'SUBJ-LAN-하위_언어',
 'r_63': 'lq8sohywze0(e_46|e_47)',
 'e_53': 'OBJ-LOC-사용_지역',
 'r_91': 'i1ypmwbj73a(e_78|e_79)',
 'e_41': 'OBJ-STU-별칭',
 'e_56': 'SUBJ-LAN-사용_집단',
 'r_117': 'i2vp76vj9q0(e_54|e_105)',
 'r_95': 't2kjowyyopl(e_78|e_83)',
 'e_78': 'SUBJ-LAN-관계_없음',
 'r_85': 'j47x4x7buab(e_76|e_79)',
 'r_69': 'vgm9n0ci2h0(e_56|e_57)',
 'e_81': 'OBJ-LOC-관계_없음',
 'e_96': 'OBJ-POH-관계_없음',
 'e_122': 'OBJ-POH-요소',
 'r_118': 'lpdm8lvlkeg(e_54|e_106)',
 'e_111': 'SUBJ-STU-연구_집단',
 'e_45': 'OBJ-DAT-시대',
 'e_52': 'SUBJ-LAN-사용_지역',
 'e_79': 'OBJ-PER-관계_없음',
 'r_90': 'mqzizeygium(e_78|e_77)',
 'r_64': '6slpea

In [7]:
#html 파일로부터 text만 추출해주는 함수
def get_context_from_html(html_file):
    html_file = re.sub(r"\n","", html_file)
    html_file = html.unescape(html_file) # 21-11-17 추가, &quot; 등 제거
    return re.findall("(<pre.+>)(.+)(</pre>)",html_file)[0][1]

In [8]:
#entity 정보가 포함된 sentence를 생성해주는 함수
def get_sentence_with_entites(subject_entity, object_entity, sentence):
    if subject_entity['start'] < object_entity['start']:
        entity1,entity2 = subject_entity, object_entity
    else:
        entity1,entity2 = object_entity, subject_entity
    
    #entity 시작 위치 및 길이 
    ett1_stt, ett1_len = entity1['start'], len(entity1['text'])
    ett2_stt, ett2_len = entity2['start'], len(entity2['text'])
    
    #문장 분리
    bf, ett1, mid, ett2, af = sentence[:ett1_stt], \
                            sentence[ett1_stt:ett1_stt+ett1_len], \
                            sentence[ett1_stt+ett1_len:ett2_stt], \
                            sentence[ett2_stt:ett2_stt+ett2_len], \
                            sentence[ett2_stt+ett2_len:]

    
    if subject_entity['start'] < object_entity['start']:
        ett1,ett2 = f"<sbj:{ett1}>", f"<obj:{ett2}>"
    else:
        ett1,ett2 = f"<obj:{ett1}>", f"<sbj:{ett2}>"
    
    return "".join([bf, ett1, mid, ett2, af])

In [9]:
import html

In [10]:
#dataframe column
# id : context title(e.g : 카카오게임, 11번가 등)
# sentence w/o entity
# sentence w entity
# subject_entity
# object_entity
# class

id_list = []
sentence_list = []
sentence_with_entities_list = []
subject_entity_list = []
object_entity_list = []
relation_list = []


# tagtog 데이터를 CSV 형태로 변경
for context_name, relation_folder, contexts_folder in zip(context_name_list, relation_folder_paths, contexts_folders_paths):
    # relation files와 context files 리스트 출력
    file_ids = [file_name.split(".ann.")[0] for file_name in os.listdir(relation_folder)]
    file_nums = [ids.split("-")[1] for ids in file_ids]
    relation_files = [relation_folder + "/"+ file_id + ".ann.json" for file_id in file_ids]
    context_files = [contexts_folder + "/"+ file_id + ".plain.html" for file_id in file_ids]
    
    #json으로 된 relation data와 html로 된 context 데이터 읽기
    for relation_file, context_file, file_num in zip(relation_files,context_files, file_nums):
        
        #subject, object 정보 추출
        with open(relation_file, "r",encoding='utf-8') as f:
            relation_json = json.load(f)
        try:
            tmp_subject, tmp_object = get_needed_relation_data(relation_json) #subject, object
            tmp_label = get_label(relation_json)
        except:
            continue
        
        #sentence, sentence with entities 정보 추출
        with open(context_file, "r",encoding='utf-8') as f:
            context_json = f.read()
        tmp_sentence = get_context_from_html(context_json)
        tmp_sentence_w_entities = get_sentence_with_entites(tmp_subject,tmp_object,tmp_sentence)
        
        #각 list에 데이터 저장
        id_list.append(f"{context_name}_{file_num}")
        sentence_list.append(tmp_sentence)
        sentence_with_entities_list.append(tmp_sentence_w_entities)
        subject_entity_list.append(tmp_subject)
        object_entity_list.append(tmp_object)
        relation_list.append(tmp_label.lower())

In [11]:
get_label(relation_json)

'STU:하위_학문'

In [12]:
import pandas as pd
column_list = ["id","sentence","sentence_with_entity","subject_entity","object_entity","class"]
data = pd.DataFrame(zip(id_list,sentence_list,sentence_with_entities_list,subject_entity_list,object_entity_list,relation_list), columns=column_list)
data

,id,sentence,sentence_with_entity,subject_entity,object_entity,class
0,동민___.csv_331,"한편, 일상 생활에서 대화에 사용되는 아랍어는, 암미야이다.","한편, 일상 생활에서 대화에 사용되는 <sbj:아랍어>는, <obj:암미야>이다.","{'start': 21, 'text': '아랍어', 'type': 'lan'}","{'start': 27, 'text': '암미야', 'type': 'lan'}",lan:하위_언어
1,동민___.csv_71,에스파냐과 지리상 가까우며 과거 프랑스와 에스파냐의 보호령이었던 모로코 북부에서도 ...,에스파냐과 지리상 가까우며 과거 프랑스와 에스파냐의 보호령이었던 <obj:모로코 북...,"{'start': 54, 'text': '에스파냐어', 'type': 'lan'}","{'start': 36, 'text': '모로코 북부', 'type': 'loc'}",lan:사용_지역
2,동민___.csv_134,"다만, 한자 및 한문이라는 공통된 문어의 기반 위에서는 중국어 제방언을 아우르는 상...","다만, 한자 및 <sbj:한문>이라는 공통된 문어의 기반 위에서는 <obj:중국어 ...","{'start': 9, 'text': '한문', 'type': 'lan'}","{'start': 31, 'text': '중국어 제방언', 'type': 'lan'}",lan:하위_언어
3,동민___.csv_93,"에스파냐어의 통시적 음운에서 주요한 특징으로는 연음화(가 가 됨), 구개음화(이 가...",<sbj:에스파냐어>의 통시적 음운에서 주요한 특징으로는 <obj:연음화>(가 가 ...,"{'start': 0, 'text': '에스파냐어', 'type': 'lan'}","{'start': 26, 'text': '연음화', 'type': 'poh'}",lan:관계_없음
4,동민___.csv_188,타이완에서는 타이완 원주민이 다른 부족과 대화할 때 일본어가 종종 쓰이는 경우가 있...,타이완에서는 <obj:타이완 원주민>이 다른 부족과 대화할 때 <sbj:일본어>가 ...,"{'start': 29, 'text': '일본어', 'type': 'lan'}","{'start': 7, 'text': '타이완 원주민', 'type': 'org'}",lan:사용_집단
...,...,...,...,...,...,...
1395,한성2________________.tsv_4,과학적 방법은 귀납적이며 경험적인 진리를 추구한다. 따라서 과학적 방법으로 얻어진 ...,과학적 방법은 귀납적이며 경험적인 진리를 추구한다. 따라서 과학적 방법으로 얻어진 ...,"{'start': 58, 'text': '수학', 'type': 'stu'}","{'start': 50, 'text': '철학', 'type': 'stu'}",stu:관계_없음
1396,한성2________________.tsv_80,화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길...,<sbj:화학>은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 ...,"{'start': 0, 'text': '화학', 'type': 'stu'}","{'start': 95, 'text': '환경오염', 'type': 'poh'}",stu:영향
1397,한성2________________.tsv_94,보일이 원래 발견자가 아님에도 보일은 1662년에 제시한 보일의 법칙으로 가장 잘 ...,<obj:보일>이 원래 발견자가 아님에도 보일은 1662년에 제시한 <sbj:보일의...,"{'start': 32, 'text': '보일의 법칙', 'type': 'stu'}","{'start': 0, 'text': '보일', 'type': 'per'}",stu:기여자
1398,한성2________________.tsv_160,물리학(物理學)과 화학의 경계에는 물리화학(物理化學)이 있고 생물학(生物學)과의 경...,<obj:물리학>(物理學)과 화학의 경계에는 <sbj:물리화학>(物理化學)이 있고 ...,"{'start': 19, 'text': '물리화학', 'type': 'stu'}","{'start': 0, 'text': '물리학', 'type': 'stu'}",stu:상위_학문


In [13]:
pd.unique(data['class'])

array(['lan:하위_언어', 'lan:사용_지역', 'lan:관계_없음', 'lan:사용_집단', 'lan:파생물',
       'lan:별칭', 'lan:상위_언어', 'stu:관계_없음', 'stu:기여자', 'stu:시대', 'stu:요소',
       'stu:영향', 'stu:상위_학문', 'stu:연구_집단', 'stu:하위_학문', 'stu:별칭'],
      dtype=object)

In [19]:
d_count = data['class'].value_counts()

In [46]:
pd.concat([d_count,d_count.apply(lambda x: str(round(x/14,2))+'%')],axis=1)

,class,class
lan:관계_없음,206,14.71%
stu:관계_없음,161,11.5%
stu:요소,136,9.71%
stu:기여자,118,8.43%
stu:하위_학문,107,7.64%
lan:사용_지역,98,7.0%
stu:상위_학문,97,6.93%
stu:영향,79,5.64%
lan:사용_집단,72,5.14%
lan:하위_언어,68,4.86%
